# c1_LPF_int

In [10]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter, freqz

In [11]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

# ADL path

In [12]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../../Dataset/SA01/D01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = [ '../../Dataset/SA01/D05_SA01_R01_1.csv',
                '../../Dataset/SA01/D05_SA01_R01_2.csv',
                '../../Dataset/SA01/D05_SA01_R02_1.csv',
                '../../Dataset/SA01/D05_SA01_R02_2.csv',
                '../../Dataset/SA01/D05_SA01_R03_1.csv',
                '../../Dataset/SA01/D05_SA01_R03_2.csv',
                '../../Dataset/SA01/D05_SA01_R04_1.csv',
                '../../Dataset/SA01/D05_SA01_R04_2.csv',
                '../../Dataset/SA01/D05_SA01_R05_1.csv',
                '../../Dataset/SA01/D05_SA01_R05_2.csv',
                '../../Dataset/SA01/D06_SA01_R01_1.csv',
                '../../Dataset/SA01/D06_SA01_R01_2.csv',
                '../../Dataset/SA01/D06_SA01_R02_1.csv',
                '../../Dataset/SA01/D06_SA01_R02_2.csv',
                '../../Dataset/SA01/D06_SA01_R03_1.csv',
                '../../Dataset/SA01/D06_SA01_R03_2.csv',
                '../../Dataset/SA01/D06_SA01_R04_1.csv',
                '../../Dataset/SA01/D06_SA01_R04_2.csv',
                '../../Dataset/SA01/D06_SA01_R05_1.csv',
                '../../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] =['../../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../../Dataset/SA01/D17_SA01_R05_2.csv']


subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23',
                'SE01', 'SE02', 'SE03', 'SE04', 'SE05',
                'SE06', 'SE07', 'SE08', 'SE09', 'SE10',
                'SE11', 'SE12', 'SE13', 'SE14', 'SE15']


total_adl_path = []

for path in paths:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_adl_path.append(path_rename)

len(total_adl_path)

3572

# ADL data 전처리

In [13]:
adl_list = []

for path in total_adl_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][200:].max()
        max_value2 = data[1][200:].max()
        max_value3 = data[2][200:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('int64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('int64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('int64')

        col_1 = col_1_lpf*col_1_lpf
        col_2 = col_2_lpf*col_2_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        adl_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

adl_df = pd.DataFrame(adl_list)
print(adl_df.shape)
row, col = adl_df.shape


target_col = row * [0]
adl_df['target'] = target_col

print(adl_df.shape)

(3209, 160)
(3209, 161)


# ADL data 저장

In [14]:
final = pd.DataFrame(adl_df)
final = final.dropna(axis=0)
final.dropna()
final = final.astype(int)

final.to_csv('../result/a_c1_LPF_int.csv', index=False)

# Fall path

In [15]:
fall_list = []

for i in range(1, 16):
    root, file = os.path.split("../../Dataset/SA01/F01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
            
print(fall_list)

subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23', 'SE06']

total_fall_path = []

for path in fall_list:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_fall_path.append(path_rename)

total_fall_path

['../../Dataset/SA01/F01_SA01_R01.csv', '../../Dataset/SA01/F01_SA01_R02.csv', '../../Dataset/SA01/F01_SA01_R03.csv', '../../Dataset/SA01/F01_SA01_R04.csv', '../../Dataset/SA01/F01_SA01_R05.csv', '../../Dataset/SA01/F02_SA01_R01.csv', '../../Dataset/SA01/F02_SA01_R02.csv', '../../Dataset/SA01/F02_SA01_R03.csv', '../../Dataset/SA01/F02_SA01_R04.csv', '../../Dataset/SA01/F02_SA01_R05.csv', '../../Dataset/SA01/F03_SA01_R01.csv', '../../Dataset/SA01/F03_SA01_R02.csv', '../../Dataset/SA01/F03_SA01_R03.csv', '../../Dataset/SA01/F03_SA01_R04.csv', '../../Dataset/SA01/F03_SA01_R05.csv', '../../Dataset/SA01/F04_SA01_R01.csv', '../../Dataset/SA01/F04_SA01_R02.csv', '../../Dataset/SA01/F04_SA01_R03.csv', '../../Dataset/SA01/F04_SA01_R04.csv', '../../Dataset/SA01/F04_SA01_R05.csv', '../../Dataset/SA01/F05_SA01_R01.csv', '../../Dataset/SA01/F05_SA01_R02.csv', '../../Dataset/SA01/F05_SA01_R03.csv', '../../Dataset/SA01/F05_SA01_R04.csv', '../../Dataset/SA01/F05_SA01_R05.csv', '../../Dataset/SA01/F06_

['../../Dataset/SA01/F01_SA01_R01.csv',
 '../../Dataset/SA02/F01_SA02_R01.csv',
 '../../Dataset/SA03/F01_SA03_R01.csv',
 '../../Dataset/SA04/F01_SA04_R01.csv',
 '../../Dataset/SA05/F01_SA05_R01.csv',
 '../../Dataset/SA06/F01_SA06_R01.csv',
 '../../Dataset/SA07/F01_SA07_R01.csv',
 '../../Dataset/SA08/F01_SA08_R01.csv',
 '../../Dataset/SA09/F01_SA09_R01.csv',
 '../../Dataset/SA10/F01_SA10_R01.csv',
 '../../Dataset/SA11/F01_SA11_R01.csv',
 '../../Dataset/SA12/F01_SA12_R01.csv',
 '../../Dataset/SA13/F01_SA13_R01.csv',
 '../../Dataset/SA14/F01_SA14_R01.csv',
 '../../Dataset/SA15/F01_SA15_R01.csv',
 '../../Dataset/SA16/F01_SA16_R01.csv',
 '../../Dataset/SA17/F01_SA17_R01.csv',
 '../../Dataset/SA18/F01_SA18_R01.csv',
 '../../Dataset/SA19/F01_SA19_R01.csv',
 '../../Dataset/SA20/F01_SA20_R01.csv',
 '../../Dataset/SA21/F01_SA21_R01.csv',
 '../../Dataset/SA22/F01_SA22_R01.csv',
 '../../Dataset/SA23/F01_SA23_R01.csv',
 '../../Dataset/SE06/F01_SE06_R01.csv',
 '../../Dataset/SA01/F01_SA01_R02.csv',


# Fall 데이터 전처리

In [16]:
fall_list = []

for path in total_fall_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][1000:].max()
        max_value2 = data[1][1000:].max()
        max_value3 = data[2][1000:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('int64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('int64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('int64')

        col_1 = col_1_lpf*col_1_lpf
        col_2 = col_2_lpf*col_2_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        fall_list.append(C_nine)
        
    except:
        pass
    
fall_df = pd.DataFrame(fall_list)
print(fall_df.shape)
row, col = fall_df.shape


target_col = row * [1]
fall_df['target'] = target_col

print(fall_df.shape)
fall_df

(1799, 160)
(1799, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.0,0.0,0.0,0.0,1.000000,2.000000,5.000000,8.000000,12.000000,17.000000,...,131.882523,127.361690,126.245792,128.085909,133.641311,141.046092,150.897316,161.400124,171.863318,1
1,0.0,0.0,0.0,0.0,1.000000,3.000000,5.099020,9.110434,14.282857,20.322401,...,185.388781,179.872177,176.195914,171.277553,167.326029,163.771182,160.853971,158.316139,156.220997,1
2,0.0,0.0,0.0,0.0,1.000000,3.162278,6.324555,10.770330,16.155494,22.472205,...,178.779193,184.480351,189.791991,195.611861,201.486972,206.237727,211.047388,216.390850,221.309738,1
3,0.0,0.0,0.0,0.0,1.000000,2.236068,5.385165,9.000000,13.453624,19.339080,...,149.566039,141.548578,134.439578,128.167859,122.478570,118.241279,114.673449,113.039816,112.254176,1
4,0.0,0.0,0.0,0.0,1.414214,3.605551,5.916080,10.344080,16.248077,23.021729,...,230.063035,238.201595,246.229568,254.530941,262.047706,268.605659,274.322803,279.495975,283.353137,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,0.0,0.0,0.0,0.0,1.000000,3.162278,6.324555,10.770330,16.155494,22.847319,...,212.435873,210.781878,209.162616,208.341067,206.799420,206.070376,204.609384,203.187106,202.328940,1
1795,0.0,0.0,0.0,0.0,1.000000,3.162278,6.324555,10.488088,15.842980,22.226111,...,253.950783,254.035431,253.007905,252.871509,251.294648,249.417321,247.016194,245.155053,242.416996,1
1796,0.0,0.0,0.0,0.0,1.000000,3.162278,6.403124,10.630146,16.093477,22.494444,...,191.253758,187.000000,182.134566,178.061787,173.507925,169.198109,165.242246,162.265215,158.971696,1
1797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Fall data 저장

In [17]:
final = pd.DataFrame(fall_df)
final = final.dropna(axis=0)
final.dropna()
final = final.astype(int)

final.to_csv('../result/f_c1_LPF_int.csv', index=False)

# All data 저장

In [18]:
adl = pd.read_csv('../result/a_c1_LPF_int.csv')
fall = pd.read_csv('../result/f_c1_LPF_int.csv')

fall_data = pd.DataFrame(fall)
print(fall_data.shape)

adl_data = pd.DataFrame(adl)
print(adl_data.shape)

file_list = [fall_data, adl_data]
result = pd.concat(file_list)
result.to_csv('../result/c1_LPF_int.csv', index=False)
print(result.shape)
result

(1768, 161)
(3108, 161)
(4876, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0,0,0,0,1,2,5,8,12,17,...,131,127,126,128,133,141,150,161,171,1
1,0,0,0,0,1,3,5,9,14,20,...,185,179,176,171,167,163,160,158,156,1
2,0,0,0,0,1,3,6,10,16,22,...,178,184,189,195,201,206,211,216,221,1
3,0,0,0,0,1,2,5,9,13,19,...,149,141,134,128,122,118,114,113,112,1
4,0,0,0,0,1,3,5,10,16,23,...,230,238,246,254,262,268,274,279,283,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,0,0,0,0,2,4,7,11,16,23,...,204,199,193,186,179,172,164,155,146,0
3104,0,0,0,0,1,3,6,10,15,23,...,262,262,262,261,260,259,259,259,257,0
3105,0,0,0,0,1,3,6,10,15,22,...,68,61,58,56,55,57,61,67,77,0
3106,0,0,0,0,1,3,6,10,16,23,...,82,76,69,62,56,50,45,41,39,0
